In [1]:
import pandas as pd
from tqdm import tqdm
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import SpectralClustering
import numpy as np
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import seaborn as sns
from ast import literal_eval
import csv
from mp_api.client import MPRester
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('data.csv')
data

,Unnamed: 0,material_id,elements,formula_pretty_reduced,formula,num_elements,chemsys,spacegroup_symbol,spacegroup_number,fav,...,0.1,0.2,0.3,0.4,0.5,0.6,1e-05,0.0001,0.001,0.01
0,0,mp-8892,"['Li', 'F', 'In']",LiInF4,Li4 In4 F16,3,F-In-Li,Pbcn,60,11.132426,...,9.190986,9.927689,9.927689,9.927689,9.927689,9.927689,0.993939,0.981707,0.962025,0.788732
1,1,mp-772968,"['Li', 'Tl', 'P', 'H', 'O']",LiTlPHO3,Li4 Tl4 P4 H4 O12,5,H-Li-O-P-Tl,C2,5,12.800916,...,11.140249,11.140249,11.140249,14.066884,14.066884,14.066884,1.000000,0.925000,0.725000,0.555556
2,2,mp-756951,"['Li', 'Sb', 'O']",LiSbO3,Li4 Sb4 O12,3,Li-O-Sb,C2/m,12,8.819405,...,4.864107,5.543978,5.543978,5.543978,5.543978,6.903721,1.000000,0.993865,0.906832,0.560606
3,3,mp-754856,"['Li', 'Y', 'Zr', 'S']",Li7Y7ZrS16,Li7 Y7 Zr1 S16,4,Li-S-Y-Zr,P2/m,10,6.245816,...,4.319241,4.319241,4.319241,4.319241,5.047983,5.047983,0.987952,0.962963,0.921569,0.735294
4,4,mp-24610,"['H', 'Li', 'O', 'P']",LiP(HO2)2,Li4 P4 H8 O16,4,H-Li-O-P,Pna2_1,33,38.676281,...,29.153401,29.153401,32.155099,32.155099,32.155099,32.155099,1.000000,0.988506,0.941860,0.537500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,868,mp-9919,"['Li', 'Zn', 'Sb']",LiZnSb,Li2 Zn2 Sb2,3,Li-Sb-Zn,P6_3mc,186,6.350430,...,4.292991,4.292991,7.108637,7.108637,7.108637,7.108637,0.987578,0.968750,0.876623,0.664286
869,869,mp-14704,"['Li', 'B', 'O', 'Y']",Li6Y(BO3)3,Li24 Y4 B12 O36,4,B-Li-O-Y,P2_1/c,14,11.867930,...,5.185502,5.979421,6.508700,7.567257,11.272210,11.272210,1.000000,0.982143,0.812500,0.742574
870,870,mp-557756,"['Li', 'Zn', 'P', 'O']",Li4Zn(PO4)2,Li16 Zn4 P8 O32,4,Li-O-P-Zn,P2_1/c,14,12.278804,...,4.985237,8.758697,8.758697,8.758697,8.758697,10.855064,1.000000,0.967480,0.811475,0.700935
871,871,mp-966801,"['Li', 'Ca', 'B', 'O']",LiCaBO3,Li8 Ca8 B8 O24,4,B-Ca-Li-O,Ibca,73,10.403979,...,5.331724,7.424524,10.040524,10.040524,10.040524,10.040524,0.989011,0.933333,0.771084,0.735294


In [3]:
missing_id = ['mp-867699', 'mp-867712', 'mp-613442', 'mp-561623', 'mp-867677', 'mp-867700']

In [4]:
for mat_id in missing_id:
    idx = data[data.material_id == mat_id].index
    data.drop(idx, inplace=True)
    data.reset_index(drop=True, inplace=True)
    
data

,Unnamed: 0,material_id,elements,formula_pretty_reduced,formula,num_elements,chemsys,spacegroup_symbol,spacegroup_number,fav,...,0.1,0.2,0.3,0.4,0.5,0.6,1e-05,0.0001,0.001,0.01
0,0,mp-8892,"['Li', 'F', 'In']",LiInF4,Li4 In4 F16,3,F-In-Li,Pbcn,60,11.132426,...,9.190986,9.927689,9.927689,9.927689,9.927689,9.927689,0.993939,0.981707,0.962025,0.788732
1,1,mp-772968,"['Li', 'Tl', 'P', 'H', 'O']",LiTlPHO3,Li4 Tl4 P4 H4 O12,5,H-Li-O-P-Tl,C2,5,12.800916,...,11.140249,11.140249,11.140249,14.066884,14.066884,14.066884,1.000000,0.925000,0.725000,0.555556
2,2,mp-756951,"['Li', 'Sb', 'O']",LiSbO3,Li4 Sb4 O12,3,Li-O-Sb,C2/m,12,8.819405,...,4.864107,5.543978,5.543978,5.543978,5.543978,6.903721,1.000000,0.993865,0.906832,0.560606
3,3,mp-754856,"['Li', 'Y', 'Zr', 'S']",Li7Y7ZrS16,Li7 Y7 Zr1 S16,4,Li-S-Y-Zr,P2/m,10,6.245816,...,4.319241,4.319241,4.319241,4.319241,5.047983,5.047983,0.987952,0.962963,0.921569,0.735294
4,4,mp-24610,"['H', 'Li', 'O', 'P']",LiP(HO2)2,Li4 P4 H8 O16,4,H-Li-O-P,Pna2_1,33,38.676281,...,29.153401,29.153401,32.155099,32.155099,32.155099,32.155099,1.000000,0.988506,0.941860,0.537500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,868,mp-9919,"['Li', 'Zn', 'Sb']",LiZnSb,Li2 Zn2 Sb2,3,Li-Sb-Zn,P6_3mc,186,6.350430,...,4.292991,4.292991,7.108637,7.108637,7.108637,7.108637,0.987578,0.968750,0.876623,0.664286
863,869,mp-14704,"['Li', 'B', 'O', 'Y']",Li6Y(BO3)3,Li24 Y4 B12 O36,4,B-Li-O-Y,P2_1/c,14,11.867930,...,5.185502,5.979421,6.508700,7.567257,11.272210,11.272210,1.000000,0.982143,0.812500,0.742574
864,870,mp-557756,"['Li', 'Zn', 'P', 'O']",Li4Zn(PO4)2,Li16 Zn4 P8 O32,4,Li-O-P-Zn,P2_1/c,14,12.278804,...,4.985237,8.758697,8.758697,8.758697,8.758697,10.855064,1.000000,0.967480,0.811475,0.700935
865,871,mp-966801,"['Li', 'Ca', 'B', 'O']",LiCaBO3,Li8 Ca8 B8 O24,4,B-Ca-Li-O,Ibca,73,10.403979,...,5.331724,7.424524,10.040524,10.040524,10.040524,10.040524,0.989011,0.933333,0.771084,0.735294


In [5]:
mp_ids = data['material_id'].tolist()

MP_API_KEY = #type your key

with MPRester(MP_API_KEY) as mpr:
    docs = [mpr.provenance.get_data_by_id(mp_id, fields =["material_id", "database_IDs", "references"]) for mp_id in mp_ids]

Retrieving ProvenanceDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ProvenanceDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ProvenanceDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ProvenanceDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ProvenanceDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

...

Retrieving ProvenanceDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ProvenanceDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ProvenanceDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ProvenanceDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ProvenanceDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
icsd_id_raw = []
for doc in docs:
    entry = doc.database_IDs.get('icsd')
    icsd_id_raw.append(entry)
    
#print(icsd_id_raw)    

icsd_id = []
for icsd in icsd_id_raw:
    if icsd is not None:
        l = [float(entry[5:]) for entry in icsd]
        icsd_id.append(l)
    else:
        icsd_id.append([np.nan])
        
print(icsd_id)

[[66693.0], [201040.0], [nan], [nan], [182309.0, 182308.0, 100200.0], [63201.0, 156893.0, 156895.0, 156899.0, 156897.0, 156901.0, 156903.0, 156889.0, 156891.0, 642175.0], [nan], [100403.0], [60935.0], [20217.0], [165212.0], [nan], [nan], [44781.0], [261236.0, 15150.0, 261238.0, 162215.0, 261239.0], [402083.0], [nan], [60850.0], [200369.0], [415201.0, 415199.0, 415200.0, 84617.0, 1585.0, 38206.0, 66708.0], [61199.0, 19007.0], [nan], [nan], [200774.0], [402341.0], [62333.0, 62878.0], [87990.0], [161498.0], [245983.0], [194777.0, 194776.0, 194778.0], [nan], [nan], [84092.0], [245964.0], [191839.0], [410043.0], [409501.0, 96940.0], [50420.0, 62301.0, 86457.0, 62298.0], [nan], [184088.0, 95979.0, 183669.0], [636.0], [165259.0, 280534.0], [nan], [174443.0, 82635.0], [245841.0], [162772.0], [109087.0], [98020.0], [168795.0], [nan], [nan], [18313.0], [65125.0], [87992.0], [415051.0], [416608.0], [9032.0], [245569.0, 95208.0], [nan], [nan], [246302.0, 67234.0], [100114.0, 616318.0], [9105.0], [

In [7]:
data.insert(5, 'icsd_id', icsd_id)

data

,Unnamed: 0,material_id,elements,formula_pretty_reduced,formula,icsd_id,num_elements,chemsys,spacegroup_symbol,spacegroup_number,...,0.1,0.2,0.3,0.4,0.5,0.6,1e-05,0.0001,0.001,0.01
0,0,mp-8892,"['Li', 'F', 'In']",LiInF4,Li4 In4 F16,[66693.0],3,F-In-Li,Pbcn,60,...,9.190986,9.927689,9.927689,9.927689,9.927689,9.927689,0.993939,0.981707,0.962025,0.788732
1,1,mp-772968,"['Li', 'Tl', 'P', 'H', 'O']",LiTlPHO3,Li4 Tl4 P4 H4 O12,[201040.0],5,H-Li-O-P-Tl,C2,5,...,11.140249,11.140249,11.140249,14.066884,14.066884,14.066884,1.000000,0.925000,0.725000,0.555556
2,2,mp-756951,"['Li', 'Sb', 'O']",LiSbO3,Li4 Sb4 O12,[nan],3,Li-O-Sb,C2/m,12,...,4.864107,5.543978,5.543978,5.543978,5.543978,6.903721,1.000000,0.993865,0.906832,0.560606
3,3,mp-754856,"['Li', 'Y', 'Zr', 'S']",Li7Y7ZrS16,Li7 Y7 Zr1 S16,[nan],4,Li-S-Y-Zr,P2/m,10,...,4.319241,4.319241,4.319241,4.319241,5.047983,5.047983,0.987952,0.962963,0.921569,0.735294
4,4,mp-24610,"['H', 'Li', 'O', 'P']",LiP(HO2)2,Li4 P4 H8 O16,"[182309.0, 182308.0, 100200.0]",4,H-Li-O-P,Pna2_1,33,...,29.153401,29.153401,32.155099,32.155099,32.155099,32.155099,1.000000,0.988506,0.941860,0.537500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,868,mp-9919,"['Li', 'Zn', 'Sb']",LiZnSb,Li2 Zn2 Sb2,"[642350.0, 42064.0]",3,Li-Sb-Zn,P6_3mc,186,...,4.292991,4.292991,7.108637,7.108637,7.108637,7.108637,0.987578,0.968750,0.876623,0.664286
863,869,mp-14704,"['Li', 'B', 'O', 'Y']",Li6Y(BO3)3,Li24 Y4 B12 O36,"[184290.0, 184288.0, 184285.0, 68653.0, 184287...",4,B-Li-O-Y,P2_1/c,14,...,5.185502,5.979421,6.508700,7.567257,11.272210,11.272210,1.000000,0.982143,0.812500,0.742574
864,870,mp-557756,"['Li', 'Zn', 'P', 'O']",Li4Zn(PO4)2,Li16 Zn4 P8 O32,[59640.0],4,Li-O-P-Zn,P2_1/c,14,...,4.985237,8.758697,8.758697,8.758697,8.758697,10.855064,1.000000,0.967480,0.811475,0.700935
865,871,mp-966801,"['Li', 'Ca', 'B', 'O']",LiCaBO3,Li8 Ca8 B8 O24,[nan],4,B-Ca-Li-O,Ibca,73,...,5.331724,7.424524,10.040524,10.040524,10.040524,10.040524,0.989011,0.933333,0.771084,0.735294


In [8]:
#Some known good examples of species with high conductivities
conductors = ['Li3N', 'Li10GeP2S12', 'L2NH', 'Li7La3Zr2O12', 'Li3ClO', 'Li3BrO', 'Li6Ps5Cl', 'Li6Ps5Br', 'Li3PS4',
              'Li7PS6', 'Li3(NH2)2', 'LiTi2(PO4)3', 'Li6PS5I', 'Li2CdCl4', 'Li6BaLa2Ta2O12', 'Li5La3Nb2O12',
              'Li7La3Zr2O12', 'Li5La3Ta2O12', 'Li3P', 'LiP5', 'Li5P4', 'Li4P3', 'Li3P7', 'Li3PO4', 'Li10SiPs2S12',
              'Li3YBr6', 'Li3ErCl6', 'Li7P3S11', 'Li3InCl6', 'LiClC3H7NO', 'LiTa2PO8', 'Li2SnS3', 'Li4SnS4', 
              'Li4PS4I', 'Li10SnP2S12', 'Li3(NH2)2I', 'Li8SiP4', 'Li8GeP4', 'Li4B7O12Cl', 'Li6BaLa2Ta2O12',
              'LiBH4']
for mat in conductors:
    idx = data.index[data['formula_pretty_reduced'] == mat]
    print(idx)

fast_ion = pd.DataFrame()
fast_ion = pd.concat([fast_ion,data.iloc[766], data.iloc[13], data.iloc[458], data.iloc[322], 
                      data.iloc[325], data.iloc[424],data.iloc[58]],axis=1)
print(data.loc[58, 'spacegroup_symbol'])
fast_ion

Index([5, 762], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([12], dtype='int64')
Index([454], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([318], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([39, 405, 487], dtype='int64')
Index([321, 618], dtype='int64')
Index([155], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([76], dtype='int64')
Index([123], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([420, 536], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')


,766,13,458,322,325,424,58
Unnamed: 0,770,14,462,326,329,428,59
material_id,mp-22899,mp-568273,mp-555089,mp-541001,mp-10485,mp-585305,mp-769763
elements,"['Li', 'I']","['Li', 'I']","['Sr', 'Li', 'Ca', 'Si', 'O']","['Li', 'In', 'I']","['Li', 'Sb', 'Ba']","['Li', 'Bi', 'P', 'O']","['Li', 'Cu', 'H', 'O']"
formula_pretty_reduced,LiI,LiI,SrLi4Ca(SiO4)2,LiInI4,BaLiSb,Li9Bi3P8O29,Li5Cu(HO2)2
formula,Li4 I4,Li2 I2,Sr4 Li16 Ca4 Si8 O32,Li4 In4 I16,Ba2 Li2 Sb2,Li18 Bi6 P16 O58,Li20 Cu4 H8 O16
...,...,...,...,...,...,...,...
0.6,5.114019,5.095564,11.447509,4.173836,2.503821,-3.948464,12.552441
1e-05,0.992248,0.984848,1.0,0.991304,1.0,1.0,1.0
0.0001,0.952756,0.969466,0.992308,0.973684,1.0,0.991525,1.0
0.001,0.893443,0.936,0.889764,0.945946,0.942857,0.803419,1.0


In [9]:
#Some known examples of species with low conductivities
conductors = ['NaCl', 'Li6PO5Cl', 'Li4GeO4', 'Li2MgBr4', 'LiBiO2', 'LiZnPS4', 'Li2TeO4', 'Li2B4O7', 'LiAlSi3O8', 'Li7SbO6',
              'Li2B3PO8', 'Li2NaBP2O8', 'Li4SiO4', 'LiPO3', 'Li2CrCl4']
for mat in conductors:
    idx = data.index[data['formula_pretty_reduced'] == mat]
    print(idx)

bad_ion = pd.DataFrame()
bad_ion = pd.concat([bad_ion,data.iloc[353], bad_ion,data.iloc[542], bad_ion,data.iloc[547], bad_ion,data.iloc[401],
                     bad_ion,data.iloc[464], bad_ion,data.iloc[587], bad_ion,data.iloc[108], bad_ion, data.iloc[827]],axis=1)
print(data.loc[387, 'spacegroup_symbol'])
bad_ion

Index([], dtype='int64')
Index([], dtype='int64')
Index([349], dtype='int64')
Index([538], dtype='int64')
Index([543], dtype='int64')
Index([397], dtype='int64')
Index([460], dtype='int64')
Index([583], dtype='int64')
Index([], dtype='int64')
Index([107], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([], dtype='int64')
Index([823], dtype='int64')
Index([350], dtype='int64')
P2_1/c


,353,542,547,401,464,587,108,827
Unnamed: 0,357,546,551,405,468,591,109,833
material_id,mp-18048,mp-6674,mp-6146,mp-643655,mp-14488,mp-17718,mp-8673,mp-4495
elements,"['Li', 'O', 'Zn', 'As']","['Li', 'O', 'Sb', 'La']","['Li', 'O', 'Zn', 'Nb']","['Li', 'Al', 'H', 'Cl', 'O']","['Li', 'O', 'P', 'In']","['Li', 'O', 'Na', 'Si', 'K', 'Cs']","['Li', 'O', 'Sn', 'Te']","['Li', 'K', 'Te']"
formula_pretty_reduced,LiZnAsO4,LiLa2SbO6,LiNbZnO4,LiAl2H6ClO6,Li3In2(PO4)3,CsKNa2Li12(SiO4)4,Li2SnTeO6,KLiTe
formula,Li18 Zn18 As18 O72,Li2 La4 Sb2 O12,Li4 Nb4 Zn4 O16,Li2 Al4 H12 Cl2 O12,Li12 In8 P12 O48,Cs2 K2 Na4 Li24 Si8 O32,Li4 Sn2 Te2 O12,K2 Li2 Te2
...,...,...,...,...,...,...,...,...
0.6,10.971363,8.400202,7.242852,10.567804,9.301846,6.930301,5.633019,2.8814
1e-05,0.991228,0.974843,1.0,0.916667,1.0,1.0,1.0,0.981308
0.0001,0.99115,0.954839,0.993976,0.8125,0.983607,1.0,0.9875,0.95283
0.001,0.927928,0.832215,0.872727,0.659574,0.727273,0.96748,0.96129,0.762376


In [10]:
fast_ion = [766, 13, 458, 322, 325, 424, 58]
slow_ion = [353, 542, 547, 401, 464, 587, 108, 827, 354]

### Using the database from "Identification of potential solid-state Li-ion conductors with semi-supervised learning"

In [11]:
data['σ(RT)(S cm-1)'] = None

In [12]:
all_labels = []
match = []

dbdf = pd.read_csv('digitized_data_for_SSEs.csv')
dbdf = dbdf.dropna(axis=0, how='any', subset=['ICSD Entry'])
for i, icsd in enumerate(tqdm(dbdf['ICSD Entry'])):
    all_labels.append(icsd)
    for j, icsd_entry in enumerate(data['icsd_id']):
        k = 0
        for entry in icsd_entry: 
            if icsd == entry:
                k = k+1
                match.append(icsd)
                if dbdf.iloc[i, 1][0] == '<':
                    data.at[j, 'σ(RT)(S cm-1)'] = float(1E-10)
                else:
                    data.at[j, 'σ(RT)(S cm-1)'] = float(dbdf.iloc[i, 1])
        if k>1:
            print('More than one value for conductivity: k='+str(k))

print('{} out of {} inputs are labeled'.format(len(match), len(all_labels)))  

100%|██████████| 346/346 [00:00<00:00, 3683.74it/s]

53 out of 346 inputs are labeled


In [13]:
mat = ['Li10Ge(PS6)2', 'Li10Si(PS6)2', 'Li6PS5I', 'Li3SbS4', 'Li3ClO']
sig = [1.20E-02, 2.30E-03, 1.30E-06, 1.50E-06, 8.50E-04]

for i, m in enumerate(mat):
    index = data.index[data['formula_pretty_reduced'] == m].tolist()
    data.at[index[0], 'σ(RT)(S cm-1)'] = sig[i]

In [14]:
data.to_csv('data_mac_labelled_using_icsd.csv')
#data.to_csv('data_pc.csv')

### Conductivity labels exploration

In [15]:
data_fast = data[data['σ(RT)(S cm-1)']>=1E-6]

data_fast

,Unnamed: 0,material_id,elements,formula_pretty_reduced,formula,icsd_id,num_elements,chemsys,spacegroup_symbol,spacegroup_number,...,0.2,0.3,0.4,0.5,0.6,1e-05,0.0001,0.001,0.01,σ(RT)(S cm-1)
12,13,mp-985585,"['Li', 'Cl', 'O']",Li3ClO,Li3 Cl1 O1,[nan],3,Cl-Li-O,Pm-3m,221,...,3.418483,3.418483,4.947412,4.947412,4.947412,1.000000,1.000000,1.012121,1.021898,0.00085
39,40,mp-18640,"['Li', 'O', 'P', 'Ti']",LiTi2(PO4)3,Li6 Ti12 P18 O72,"[184088.0, 95979.0, 183669.0]",4,Li-O-P-Ti,R-3c,167,...,2.657019,2.657019,2.657019,2.657019,2.657019,0.984252,0.897638,0.661290,0.495238,0.000008
64,65,mp-22983,"['Li', 'Al', 'Cl']",LiAlCl4,Li4 Al4 Cl16,"[1040.0, 35277.0, 35276.0, 35275.0]",3,Al-Cl-Li,P2_1/c,14,...,4.537040,7.293993,7.293993,7.293993,7.293993,1.000000,0.990291,0.901961,0.604396,0.000001
76,77,mp-736,"['Li', 'P']",Li3P,Li6 P2,"[642223.0, 26880.0, 240861.0]",2,Li-P,P6_3/mmc,194,...,6.583336,10.191511,10.951127,14.464349,14.464349,0.993902,0.993789,0.993590,1.000000,0.000703
186,187,mp-29151,"['Li', 'N', 'Cl']",Li5NCl2,Li15 N3 Cl6,[84763.0],3,Cl-Li-N,R-3m,166,...,3.021047,5.610517,5.610517,6.257884,6.257884,1.000000,1.000000,1.000000,0.943662,0.000001
265,267,mp-696129,"['Li', 'Si', 'P', 'S']",Li10Si(PS6)2,Li20 Si2 P4 S24,[nan],4,Li-P-S-Si,P4_2mc,105,...,-2.550960,4.402649,5.012615,5.012615,5.012615,1.000000,1.000000,1.000000,1.000000,0.0023
267,269,mp-696128,"['Li', 'Ge', 'P', 'S']",Li10Ge(PS6)2,Li20 Ge2 P4 S24,[nan],4,Ge-Li-P-S,P4_2mc,105,...,-2.276746,3.993160,4.716610,4.716610,4.716610,1.000000,1.000000,1.000000,0.978571,0.012
321,325,mp-985582,"['Li', 'P', 'S', 'I']",Li6PS5I,Li24 P4 S20 I4,[nan],4,I-Li-P-S,F-43m,216,...,-4.857220,-4.857220,-4.857220,-4.857220,-4.857220,1.000000,1.000000,0.955752,0.709677,0.000001
454,458,mp-28593,"['Li', 'O', 'Br']",Li3BrO,Li3 Br1 O1,[67265.0],3,Br-Li-O,Pm-3m,221,...,5.891858,5.891858,5.891858,10.789909,10.789909,1.000000,0.986928,0.972973,0.880597,0.000001
554,558,mp-756316,"['Li', 'Sb', 'S']",Li3SbS4,Li6 Sb2 S8,[nan],3,Li-S-Sb,Pmn2_1,31,...,8.999829,8.999829,8.999829,9.154564,10.005611,1.000000,0.976923,0.944444,0.750000,0.000002


In [16]:
print('Number of fast-ion conductors: '+str(len(data_fast)))

Number of fast-ion conductors: 13


In [17]:
data_slow = data[data['σ(RT)(S cm-1)']<1E-6]

data_slow

,Unnamed: 0,material_id,elements,formula_pretty_reduced,formula,icsd_id,num_elements,chemsys,spacegroup_symbol,spacegroup_number,...,0.2,0.3,0.4,0.5,0.6,1e-05,0.0001,0.001,0.01,σ(RT)(S cm-1)
52,53,mp-18147,"['Li', 'O', 'Si', 'Ga']",LiGaSiO4,Li18 Ga18 Si18 O72,[65125.0],4,Ga-Li-O-Si,R3,146,...,9.856570,10.411002,10.411002,12.813541,13.367973,1.000000,0.958678,0.822034,0.633028,0.0
62,63,mp-27275,"['Li', 'B', 'O']",Li3BO3,Li12 B4 O12,[9105.0],3,B-Li-O,P2_1/c,14,...,8.601866,8.601866,10.340474,12.575828,12.575828,0.990991,0.963636,0.861111,0.711340,0.0
100,101,mp-4156,"['Li', 'O', 'Zr']",Li2ZrO3,Li8 Zr4 O12,"[94897.0, 94896.0, 94894.0, 291154.0, 94893.0,...",3,Li-O-Zr,C2/c,15,...,3.718460,3.718460,6.503348,6.503348,6.503348,1.000000,1.000000,0.981928,0.776119,0.0
117,118,mp-558105,"['Li', 'B', 'O', 'F']",LiB6O9F,Li4 B24 O36 F4,[420286.0],4,B-F-Li-O,Pna2_1,33,...,8.623060,9.196701,9.196701,9.196701,10.917621,0.992248,0.921875,0.566929,0.236842,0.0
129,130,mp-23497,"['Li', 'Zn', 'I']",Li2ZnI4,Li8 Zn4 I16,"[69034.0, 402062.0]",3,I-Li-Zn,Pnma,62,...,4.389673,4.389673,4.389673,4.389673,4.389673,0.985612,0.971014,0.924812,0.719008,0.0
150,151,mp-3635,"['Li', 'B', 'O']",LiBO2,Li4 B4 O8,"[37060.0, 200891.0, 16568.0]",3,B-Li-O,P2_1/c,14,...,10.344364,10.344364,10.344364,12.039833,12.039833,0.993243,0.952381,0.561644,0.422222,0.0
184,185,mp-5840,"['Li', 'O', 'Sc']",LiScO2,Li4 Sc4 O8,"[31316.0, 36124.0]",3,Li-O-Sc,I4_1/amd,141,...,3.746910,3.746910,11.284811,11.284811,11.284811,0.993976,0.975460,0.955128,0.778626,0.0
229,231,mp-8926,"['Li', 'B', 'N']",Li3BN2,Li6 B2 N4,"[155128.0, 71085.0, 655673.0]",3,B-Li-N,P4_2/mnm,136,...,3.690109,3.690109,3.690109,3.690109,12.190777,1.000000,1.000000,0.945205,0.916667,0.0
255,257,mp-7535,"['Li', 'O', 'Al']",Li5AlO4,Li10 Al2 O8,"[16229.0, 1037.0]",3,Al-Li-O,Pmmn,59,...,5.807534,6.398257,7.875064,7.875064,8.465787,1.000000,0.987342,0.973856,0.841727,0.0
275,277,mp-4657,"['Li', 'O', 'Ga']",Li5GaO4,Li40 Ga8 O32,"[16926.0, 9082.0]",3,Ga-Li-O,Pbca,61,...,6.000244,6.794394,8.117977,9.441560,9.441560,0.993789,0.987421,0.974026,0.877698,0.0


In [18]:
print('Number of labelled data: '+str(len(data_fast)+len(data_slow)))

Number of labelled data: 56


In [19]:
data['Conductor?'] = None

In [20]:
for idx in data_fast.index.values:
    data.at[idx, 'Conductor?'] = True
for idx in data_slow.index.values:
    data.at[idx, 'Conductor?'] = False
    
data

,Unnamed: 0,material_id,elements,formula_pretty_reduced,formula,icsd_id,num_elements,chemsys,spacegroup_symbol,spacegroup_number,...,0.3,0.4,0.5,0.6,1e-05,0.0001,0.001,0.01,σ(RT)(S cm-1),Conductor?
0,0,mp-8892,"['Li', 'F', 'In']",LiInF4,Li4 In4 F16,[66693.0],3,F-In-Li,Pbcn,60,...,9.927689,9.927689,9.927689,9.927689,0.993939,0.981707,0.962025,0.788732,None,None
1,1,mp-772968,"['Li', 'Tl', 'P', 'H', 'O']",LiTlPHO3,Li4 Tl4 P4 H4 O12,[201040.0],5,H-Li-O-P-Tl,C2,5,...,11.140249,14.066884,14.066884,14.066884,1.000000,0.925000,0.725000,0.555556,None,None
2,2,mp-756951,"['Li', 'Sb', 'O']",LiSbO3,Li4 Sb4 O12,[nan],3,Li-O-Sb,C2/m,12,...,5.543978,5.543978,5.543978,6.903721,1.000000,0.993865,0.906832,0.560606,None,None
3,3,mp-754856,"['Li', 'Y', 'Zr', 'S']",Li7Y7ZrS16,Li7 Y7 Zr1 S16,[nan],4,Li-S-Y-Zr,P2/m,10,...,4.319241,4.319241,5.047983,5.047983,0.987952,0.962963,0.921569,0.735294,None,None
4,4,mp-24610,"['H', 'Li', 'O', 'P']",LiP(HO2)2,Li4 P4 H8 O16,"[182309.0, 182308.0, 100200.0]",4,H-Li-O-P,Pna2_1,33,...,32.155099,32.155099,32.155099,32.155099,1.000000,0.988506,0.941860,0.537500,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,868,mp-9919,"['Li', 'Zn', 'Sb']",LiZnSb,Li2 Zn2 Sb2,"[642350.0, 42064.0]",3,Li-Sb-Zn,P6_3mc,186,...,7.108637,7.108637,7.108637,7.108637,0.987578,0.968750,0.876623,0.664286,None,None
863,869,mp-14704,"['Li', 'B', 'O', 'Y']",Li6Y(BO3)3,Li24 Y4 B12 O36,"[184290.0, 184288.0, 184285.0, 68653.0, 184287...",4,B-Li-O-Y,P2_1/c,14,...,6.508700,7.567257,11.272210,11.272210,1.000000,0.982143,0.812500,0.742574,None,None
864,870,mp-557756,"['Li', 'Zn', 'P', 'O']",Li4Zn(PO4)2,Li16 Zn4 P8 O32,[59640.0],4,Li-O-P-Zn,P2_1/c,14,...,8.758697,8.758697,8.758697,10.855064,1.000000,0.967480,0.811475,0.700935,None,None
865,871,mp-966801,"['Li', 'Ca', 'B', 'O']",LiCaBO3,Li8 Ca8 B8 O24,[nan],4,B-Ca-Li-O,Ibca,73,...,10.040524,10.040524,10.040524,10.040524,0.989011,0.933333,0.771084,0.735294,None,None


In [21]:
data.to_csv('data_labelled.csv')